In [1]:
import inspect

import pandas as pd
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples, PairwiseMetric
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory
from vertexai.preview.evaluation import notebook_utils

In [6]:

instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]


eval_dataset = pd.DataFrame(
    {
        "context": context,
        "instruction": [instruction] * len(context),
        "reference": reference,
    }
)

In [ ]:
prompt_template = "{instruction}. Article: {context}. Summary:"

In [25]:
metric_name = "pairwise_text_quality"

EXPERIMENT_NAME = "model-selection"

pairwise_text_quality_result = EvalTask(
    dataset=eval_dataset,
    metrics=[
        PairwiseMetric(
            metric=metric_name,
            metric_prompt_template=MetricPromptTemplateExamples.get_prompt_template(
                metric_name
            ),
            # Baseline model for pairwise comparison
            baseline_model=GenerativeModel(
                "gemini-2.0-flash",
            ),
        ),
    ],
    experiment=EXPERIMENT_NAME,
).evaluate(
    # Specify candidate model for pairwise comparison
    model=GenerativeModel(
        "gemini-2.5-flash-preview-04-17"
    ),
    prompt_template=prompt_template,
)

Associating projects/234439745674/locations/us-central1/metadataStores/default/contexts/model-selection-ae743e50-e5dc-4eb9-b09f-70a323fb3301 to Experiment: model-selection


Logging Eval Experiment metadata: {'prompt_template': '{instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-2.5-flash-preview-04-17'}
Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
Generating a total of 5 responses from Gemini model gemini-2.5-flash-preview-04-17.


100%|██████████| 5/5 [00:04<00:00,  1.15it/s]

All 5 responses are successfully generated from Gemini model gemini-2.5-flash-preview-04-17.
Multithreaded Batch Inference took: 4.363047083003039 seconds.
Generating a total of 5 responses from Gemini model gemini-2.0-flash.



100%|██████████| 5/5 [00:01<00:00,  2.82it/s]

All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
Multithreaded Batch Inference took: 1.7743426669985638 seconds.
Computing metrics with a total of 5 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 5/5 [00:05<00:00,  1.06s/it]

All 5 metric requests are successfully computed.
Evaluation Took:5.2864889159973245 seconds


In [26]:
notebook_utils.display_eval_result(
    title="Side-by-side Eval Results", eval_result=pairwise_text_quality_result
)

## Side-by-side Eval Results

### Summary Metrics

,row_count,pairwise_text_quality/candidate_model_win_rate,pairwise_text_quality/baseline_model_win_rate
0,5.0,0.8,0.2


### Row-based Metrics

,context,instruction,reference,prompt,response,baseline_model_response,pairwise_text_quality/explanation,pairwise_text_quality/pairwise_choice
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Summarize the following article. Article: To m...,Here's a summary of the article:\n\nThis artic...,Classic spaghetti carbonara involves cooking p...,Both responses followed instructions. CANDIDAT...,CANDIDATE
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Summarize the following article. Article: Prep...,This article provides instructions for making ...,"To make perfect risotto, slowly cook onions an...",BASELINE response includes the main point of t...,BASELINE
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Summarize the following article. Article: For ...,"To grill a flavorful steak, choose a well-marb...","To grill a delicious steak, choose a well-marb...",Both responses follow the instructions of the ...,CANDIDATE
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Summarize the following article. Article: Crea...,This article provides a simple recipe for maki...,Homemade tomato soup is easily made by sautéin...,CANDIDATE response is more concise and directl...,CANDIDATE
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Summarize the following article. Article: To b...,"To bake a chocolate cake from scratch, first p...","To bake a homemade chocolate cake, preheat the...",CANDIDATE response is better because it is mor...,CANDIDATE


In [33]:

notebook_utils.display_explanations(
    pairwise_text_quality_result, metrics=[metric_name], num=1
)

In [32]:
candidate_model_win_rate = round(
    pairwise_text_quality_result.summary_metrics[
        f"{metric_name}/candidate_model_win_rate"
    ]
    * 100
)
print(
    f"Win rate: Autorater prefers Candidate Model over Baseline Model {candidate_model_win_rate}% of time."
)


Win rate: Autorater prefers Candidate Model over Baseline Model 80% of time.
